# Learning Python as an R user

In [97]:
# Introduction

In [98]:
# what are your goals for this post? Are you going to explain every detail?

# Import libraries

In [99]:
# did you install these libraries somewhere? what does import do? what do the dots "." mean?

In [100]:
import pandas
import math
import numpy
import plotly.express
import plotly.offline
import scipy
import statsmodels.api
import statsmodels.formula.api
import patsy.contrasts

## Offline Plotting Settings

In [101]:
# what do this code mean? why do you have to start with "plotly"?

In [102]:
plotly.offline.init_notebook_mode(connected = True)

# Generate data
## `group` variable with 4 levels

In [103]:
# similar to the `base::rep` function in R. [] make lists. what are lists?

In [104]:
group = numpy.repeat(["A", "B", "C", "D"], repeats = 25)

## Save parameters

In [105]:
# () make tuples. what are tuples?

In [106]:
mean = (3, 4, 4, 3)
sigma = (1, 1, 1, 1)
n = (25, 25, 25, 25)

## Random normal variable `x` whose means depend on levels of `group` variable

In [ ]:
# what are list comprehensions? why do you have to concatenate it?

In [107]:
# Loop through equal-length mean, sigma, and n (group size)
x = numpy.concatenate(
    [numpy.random.normal(loc = i, scale = j, size = k) for (i, j, k) in zip(mean, sigma, n)]
)

## Variable `y` correlated r = 0.75 with x

In [108]:
y = x * 0.75 + numpy.random.normal(loc = 0, scale = 1, size = sum(n))

## Store variables in a data frame

In [ ]:
# why the colons? why the squigly brackets?

In [109]:
data1 = pandas.DataFrame({"y": y, "x": x, "group": group})

## Create contrast variables for use in linear regression on `group` variable

In [110]:
# Helmert contrasts
group_helmert = patsy.contrasts.Helmert().code_without_intercept(list(set(group)))

# 2 main effects and 1 interaction contrast
group_factorial = patsy.contrasts.ContrastMatrix([[-1, -1, 1], [-1, 1, -1], [1, -1, -1], [1, 1, 1]], 
                                                 ["Main Effect 1", "Main Effect 2", "Interaction"])

## Add contrast variables to data frame

In [162]:
data1.loc[data1["group"] == "A", "main1"] = -1
data1.loc[data1["group"] == "B", "main1"] = -1
data1.loc[data1["group"] == "C", "main1"] = 1
data1.loc[data1["group"] == "D", "main1"] = 1

data1.loc[data1["group"] == "A", "main2"] = -1
data1.loc[data1["group"] == "B", "main2"] = 1
data1.loc[data1["group"] == "C", "main2"] = -1
data1.loc[data1["group"] == "D", "main2"] = 1

data1.loc[data1["group"] == "A", "interaction"] = 1
data1.loc[data1["group"] == "B", "interaction"] = -1
data1.loc[data1["group"] == "C", "interaction"] = -1
data1.loc[data1["group"] == "D", "interaction"] = 1

TypeError: '>' not supported between instances of 'list' and 'int'

# Plots
## Boxplots

In [112]:
# boxplots of y by group
plotly.offline.iplot(
    plotly.express.box(data1, x = "group", y = "y")
)

In [113]:
# boxplots of x by group
plotly.offline.iplot(
    plotly.express.box(data1, x = "group", y = "x")
)

## Histograms

In [114]:
# y histograms by group
plotly.offline.iplot(
    plotly.express.histogram(data1, x = "y", facet_col = "group")
)

In [115]:
# x histograms by group
plotly.offline.iplot(
    plotly.express.histogram(data1, x = "x", facet_col = "group")
)

## Scatterplot

In [116]:
plotly.offline.iplot(
    plotly.express.scatter(data1, x = "x", y = "y", color = "group", trendline = "ols", facet_col = "group")
)

## Bars of `group` means with 95% confidence intervals

In [117]:
group_desc = data1.groupby("group")["y"].agg(["mean", "sem", "count"]).reset_index()
group_desc["df"] = group_desc["count"] - 1
group_desc["lower"] = group_desc["mean"] - scipy.stats.t.ppf(1 - 0.05 / 2, df = group_desc["df"]) * group_desc["sem"]
group_desc["upper"] = group_desc["mean"] + scipy.stats.t.ppf(1 - 0.05 / 2, df = group_desc["df"]) * group_desc["sem"]

# Plot
plotly.offline.iplot(
    plotly.express.bar(group_desc, x = "group", y = "mean", error_y_minus = "lower", error_y = "upper", color = "group")
)

# Descriptive Statistics

In [118]:
data1.groupby("group")[["x", "y"]].describe().round(2)

x                                               y                    \
      count  mean   std   min   25%   50%   75%   max count  mean   std   min   
group                                                                           
A      25.0  2.92  0.79  1.13  2.37  2.83  3.34  4.78  25.0  2.41  0.99  0.21   
B      25.0  4.26  0.94  2.67  3.49  4.40  4.89  6.21  25.0  3.47  1.08  1.37   
C      25.0  3.69  1.05  1.76  3.13  3.84  4.30  5.78  25.0  2.96  1.33  0.31   
D      25.0  3.11  0.81  1.70  2.40  3.46  3.62  4.87  25.0  2.16  1.30 -0.23   

                               
        25%   50%   75%   max  
group                          
A      1.91  2.50  3.01  4.19  
B      2.88  3.62  4.31  5.06  
C      2.10  2.75  4.04  6.01  
D      1.17  2.16  3.25  4.12

# Correlation

In [119]:
# Save r and p-value
r1, pvalue1 = scipy.stats.pearsonr(x, y)

# Save degrees of freedom
ddf1 = len(x) - 2

# Compute t-statistic and degress of freedom, isf for upper tail of t distribution
t1 = scipy.stats.t.isf(pvalue1 / 2, df = ddf1)

# Compute standard error
se1 = r1 / t1

# Compute lower and upper confidence intervals
lower1, upper1 = (numpy.tanh(numpy.arctanh(r1) - 1 / math.sqrt(len(x) - 3) * scipy.stats.norm.ppf(1 - 0.05 / 2)),
                  numpy.tanh(numpy.arctanh(r1) + 1 / math.sqrt(len(x) - 3) * scipy.stats.norm.ppf(1 - 0.05 / 2)))

# Print most basic results
"r = {0}, 95%CI [{1}, {2}], t({3}) = {4}, p = {5}".format(r1.round(2), lower1.round(2), upper1.round(2), ddf1, t1.round(2), pvalue1.round(3))

'r = 0.6, 95%CI [0.46, 0.71], t(98) = 7.43, p = 0.0'

# Analyses
## Regression
### Fit linear regression

In [120]:
# regress y on numeric/continuous x
ols_fit1 = statsmodels.formula.api.ols("y ~ x", data = data1).fit()

### Results Summary

In [121]:
ols_fit1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     55.19
Date:                Mon, 13 Jan 2020   Prob (F-statistic):           4.10e-11
Time:                        20:53:27   Log-Likelihood:                -143.11
No. Observations:                 100   AIC:                             290.2
Df Residuals:                      98   BIC:                             295.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1772      0.361      0.491      0.625      -0.539       0.893
x              0.7362      0.099      7.429      0.000       0.540       0.933
==============================================================================
Omnibus:                        0.411   Durbin-Watson:                   1.775
Prob(Omnibus):                  0.814   Jarque-Bera (JB):                0.233
Skew:                          -0.117   Prob(JB):                        0.890
Kurtosis:                       3.032   Cond. No.                         13.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Fit linear regression

In [122]:
# helmert contrasts on group
ols_fit2 = statsmodels.formula.api.ols("y ~ C(group, group_helmert)", data = data1).fit()

### Results Summary

In [123]:
ols_fit2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     6.140
Date:                Mon, 13 Jan 2020   Prob (F-statistic):           0.000729
Time:                        20:53:27   Log-Likelihood:                -156.67
No. Observations:                 100   AIC:                             321.3
Df Residuals:                      96   BIC:                             331.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        2.7488      0.118     23.233      0.000       2.514       2.984
C(group, group_helmert)[H.B]     0.5302      0.167      3.169      0.002       0.198       0.862
C(group, group_helmert)[H.A]     0.0049      0.097      0.050      0.960      -0.187       0.197
C(group, group_helmert)[H.C]    -0.1977      0.068     -2.894      0.005      -0.333      -0.062
==============================================================================
Omnibus:                        1.442   Durbin-Watson:                   1.824
Prob(Omnibus):                  0.486   Jarque-Bera (JB):                1.235
Skew:                          -0.089   Prob(JB):                        0.539
Kurtosis:                       2.486   Cond. No.                         2.45
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [124]:
# factorial contrasts (2 main effects and 1 interaction)
ols_fit3 = statsmodels.formula.api.ols("y ~ C(group, group_factorial)", data = data1).fit()

## Results Summary

In [125]:
ols_fit3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     6.140
Date:                Mon, 13 Jan 2020   Prob (F-statistic):           0.000729
Time:                        20:53:27   Log-Likelihood:                -156.67
No. Observations:                 100   AIC:                             321.3
Df Residuals:                      96   BIC:                             331.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  2.7488      0.118     23.233      0.000       2.514       2.984
C(group, group_factorial)Main Effect 1    -0.1928      0.118     -1.630      0.106      -0.428       0.042
C(group, group_factorial)Main Effect 2     0.0650      0.118      0.549      0.584      -0.170       0.300
C(group, group_factorial)Interaction      -0.4652      0.118     -3.932      0.000      -0.700      -0.230
==============================================================================
Omnibus:                        1.442   Durbin-Watson:                   1.824
Prob(Omnibus):                  0.486   Jarque-Bera (JB):                1.235
Skew:                          -0.089   Prob(JB):                        0.539
Kurtosis:                       2.486   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Fit linear regression

In [126]:
ols_fit4 = statsmodels.formula.api.ols("y ~ main1 + main2 + interaction", data = data1).fit()

## Results Summary

In [127]:
ols_fit4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     6.140
Date:                Mon, 13 Jan 2020   Prob (F-statistic):           0.000729
Time:                        20:53:27   Log-Likelihood:                -156.67
No. Observations:                 100   AIC:                             321.3
Df Residuals:                      96   BIC:                             331.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       2.7488      0.118     23.233      0.000       2.514       2.984
main1          -0.1928      0.118     -1.630      0.106      -0.428       0.042
main2           0.0650      0.118      0.549      0.584      -0.170       0.300
interaction    -0.4652      0.118     -3.932      0.000      -0.700      -0.230
==============================================================================
Omnibus:                        1.442   Durbin-Watson:                   1.824
Prob(Omnibus):                  0.486   Jarque-Bera (JB):                1.235
Skew:                          -0.089   Prob(JB):                        0.539
Kurtosis:                       2.486   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""